In [161]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador, CVM,Bawm
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
import numpy as np

In [162]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1()
cvm = CVM()
bawm = Bawm()

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = fdt.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Data_ini_boletas
data_ini = pd.datetime(2024,1,1)

#Produtos
produtos = crm.rating_produtos()[['Ativo','ISIN','CETIP','emissor']]

In [163]:
#retona a lista de cotista

posicao_cotistas = posicao.posicao_total_depara(hoje,)
cotistas = posicao_cotistas[(posicao_cotistas['Classe']=='F Exclusivo')|(posicao_cotistas['Classe']=='F Exclusivo Ações')]
cotistas = cotistas[['NomeContaCRM','IdBDS','FinanceiroFinal']].drop_duplicates().rename(columns={'FinanceiroFinal':'Financeiro_Cotista','NomeContaCRM':'Cotista'})

In [164]:
#Retorna a carteira dos fundos que estão encerrando

fd_encerramento = pd.read_excel('C:/Temp/Foundation/Encerramento/encerramento_fundos.xlsx')

po_cadastro = posicao.po_cadastro_all()
#retorna os perfis por cotista
perfil = po_cadastro[po_cadastro['Tipo']=='Cliente'][['GuidContaCRM','Perfil','Segmento']]

#retorna só os campos necessários para fazer o merger
po_cadastro['CNPJ'] = po_cadastro['CNPJ'].astype('str')
po_cadastro = po_cadastro[['GuidContaCRM','NomeContaCRM','Officer','Controller','DPM','DeputyDPM','CNPJ','Administrador','Custodiante','IdBDS']]

po_posicao = posicao.posicao_fundos_all(hoje).drop(columns=['IdBDS'])

#Retorna as informações da Po_Ativos
po_ativos = posicao.ativos_cadastro_po_ativos().reset_index()[['GuidAtivo','TipoProduto']]

#Atualiza a posição com os tipos por produto da PO_Ativos
po_posicao = pd.merge(left=po_posicao,right=po_ativos,left_on='GuidProduto',right_on='GuidAtivo',how='left').drop(columns=['GuidAtivo'])

#Incluir a po_posição os dados cadastrais de cada fundo

po_posicao = pd.merge(left=po_posicao,right=po_cadastro,on='NomeContaCRM',how='left')

#rertorna os cotistas dos fundos

po_posicao = pd.merge(left=po_posicao,right=cotistas, on='IdBDS',how='left').drop(columns=['IdBDS'])

#retona o quanto cotista tem em cada fundo
pls = posicao.fundos_pl_estimado()[['NomeContaCRM','PL']]
po_posicao = pd.merge(left=po_posicao,right=pls, on='NomeContaCRM',how='left').rename(columns={'GuidContaCRM_x':'GuidContaCRM'})
po_posicao['% do Fundo'] = po_posicao['Financeiro_Cotista'] / po_posicao['PL']

#Retorna Estrutura ideal de contas

contas = crm.contas_movimento_all()[['accountid','new_titularidadeidname','Banco','new_agencia','new_numeroconta','new_digitoconta','ControleJBBName','NomeContaMovimento']]
contas = contas[(contas['new_agencia']=='2856')|(contas['Banco']=='Banco BTG Pactual SA')].rename(columns={'accountid':'GuidContaCRM'})
contas['Possui Estrutura de Contas'] = 'Sim'
contas['Conta_BTG'] = np.nan
contas['Conta_BTG'].loc[(contas['Banco']=='Banco BTG Pactual SA')]='ok para privados'
contas = contas[['new_titularidadeidname','GuidContaCRM','Possui Estrutura de Contas','Conta_BTG']]
contas = pd.merge(left=contas,right=perfil,on='GuidContaCRM',how='left')
contas = contas.drop_duplicates()


po_posicao = po_posicao.drop_duplicates(subset=['NomeContaCRM','NomeSuperCarteiraCRM','FinanceiroFinal','GuidProduto','Cotista','Financeiro_Cotista',])

#Verifica se os clientes possuem a estrutura ideal de contas

po_posicao = pd.merge(left=po_posicao,right=contas,left_on='Cotista',right_on='new_titularidadeidname',how='left').rename(columns={'TipoProduto_y':'TipoProduto','GuidContaCRM_x':'GuidContaCRM'})


In [165]:
#Retorna os emails dos respom

usuarios = posicao.po_cadastro_all()[['NomeContaCRM','Officer','Controller','DPM','OfficerUsuarioU','DPMUsuarioU','DeputyDPMUsuarioU','ControllerUsuarioU']]
emails_usuarios = crm.lista_usuarios_u()
carteiras = po_posicao[['NomeContaCRM']].drop_duplicates()
carteiras = pd.merge(left=carteiras, right=usuarios, on='NomeContaCRM', how='left')

emails_envio = []

#emails_usuarios['fullname']=emails_usuarios['fullname'].str.lower()
for idx, row in carteiras.iterrows(): 
    lista_email=[]
    if row['OfficerUsuarioU'] in emails_usuarios['UsuarioU'].to_list():  
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']== row['OfficerUsuarioU']].iat[0,2])
    if row['DPMUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['DPMUsuarioU']].iat[0,2])
    if row['DeputyDPMUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['DeputyDPMUsuarioU']].iat[0,2])
    if row['ControllerUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['ControllerUsuarioU']].iat[0,2]) 
    emails_envio.append(lista_email)
carteiras['emails_aprovadores']= emails_envio   
carteiras = carteiras[['NomeContaCRM','emails_aprovadores']]

In [166]:
def acao_pm(NomeProduto,FormaCondominio,Disponivel_Secundario,Movimentacao_Solicitada,TipoProduto):
    ativos_simples = ['NTN-B','NTN-C','NTN-F','LTN','LFT','AÇOES','FUT','BOLSA','NTN-B']
    produto = NomeProduto.split()
    if Movimentacao_Solicitada == 'Sim':
        acao=np.nan
    if FormaCondominio != 'Aberto' and 'FIDC' in produto:
        acao = 'Disponibilizar no secundário'
    elif FormaCondominio == 'Fechado' and 'FIP' in produto :
        acao = 'Verificar se podemos passar para PF'
    elif TipoProduto != 'Cotas' and  Disponivel_Secundario == np.nan:
        acao = 'Disponibilizar no secundário'
    elif TipoProduto == 'Cotas' and  FormaCondominio == 'Fechado':
        acao ='Verificar se podemos passar para PF'
    elif (TipoProduto == 'COTAS' and  FormaCondominio == 'Aberto') or (TipoProduto == 'BOLSA') or (TipoProduto == 'FUT') or TipoProduto in ativos_simples :
        acao ='Resgatar / Vender' 
    else:
        acao = 'Disponibilizar no secundário'
    return acao    
        
        

        

In [167]:
ativos_desconsiderar = ['COMP','CAIXA','Ajustes']
ativos_simples = ['NTN-B','NTN-C','NTN-F','LTN','LFT','AÇOES','FUT','BOLSA','NTN-B']

fundos = list(fd_encerramento['CNPJ'])
fundos = [x.replace('-','').replace('.','').replace('/','')   for x in fundos ]

po_cadastro = po_cadastro.loc[po_cadastro['CNPJ'].isin(fundos)][['NomeContaCRM']]
po_cadastro = list(po_cadastro['NomeContaCRM'])

pos_fd_encerramento = po_posicao.loc[~po_posicao['TipoProduto'].isin(ativos_desconsiderar)]
pos_fd_encerramento = pos_fd_encerramento.loc[pos_fd_encerramento['NomeContaCRM'].isin(po_cadastro)]
pos_fd_encerramento['GuidContaCRM'] = pos_fd_encerramento['GuidContaCRM'].str.lower()
guids = list(pos_fd_encerramento['GuidContaCRM'])
pos_fd_encerramento['Disponivel_Secundario'] = pos_fd_encerramento.apply(lambda row : row['QtdeBloq'] if row['TipoProduto'] not in ativos_simples else np.nan,axis=1 )
pos_fd_encerramento = pos_fd_encerramento[['DataArquivo','GuidContaCRM','NomeContaCRM','TipoProduto','Classe','Conta_BTG','GuidProduto','NomeProduto','CodigoCetip','FormaCondominio','QuantidadeFinal','QtdeBloq','Disponivel_Secundario','FinanceiroFinal','ResgPedido','ResgCot','ResgFin','Officer','Controller','DPM','DeputyDPM','CNPJ','Segmento','Administrador','Custodiante','Cotista','Perfil','Possui Estrutura de Contas','% do Fundo']].rename(columns={'QtdeBloq':'QtdeBloq_Margem'})


In [168]:
boletas = bol.boletas_range_datas(data_ini=data_ini, data_fim=hoje, exportado=True, deletado=False).reset_index()
boletas = boletas[(boletas['StatusCRM']!='Cancelada')&(boletas['StatusCRM']!='Devolvida')]
boletas = boletas[boletas['DataCot']>hoje]
boletas['ContaCRMGuid'] = boletas['ContaCRMGuid'].str.lower()
guids_v2 = [x.replace('{','').replace('}','') for x in boletas['ContaCRMGuid']]
boletas['ContaCRMGuid'] = guids_v2
boletas = boletas.loc[boletas['ContaCRMGuid'].isin(guids)]
boletas = boletas[boletas['TipoMov']=='V']
boletas['Movimentacao_Solicitada'] = 'Sim'
boletas = boletas[['ContaCRMGuid','AtivoGuid','Movimentacao_Solicitada','TipoMov','ResgTot','DataMov','DataCot','DataFin','StatusCRM']]

#INcluir nas informações de posição
verificacao = pd.merge(left = pos_fd_encerramento,right= boletas,left_on=['GuidContaCRM','GuidProduto'],right_on=['ContaCRMGuid','AtivoGuid'], how='outer')
verificacao = verificacao[verificacao['DataArquivo'].notnull()]
verificacao = verificacao[verificacao['Classe']!='Ajuste']
verificacao = verificacao.sort_values(['NomeContaCRM','NomeProduto','ResgTot'],ascending= [True,True,False])
#verificacao = verificacao[['DataArquivo','NomeContaCRM','TipoProduto','NomeProduto','Classe','CodigoCetip','QuantidadeFinal','QtdeBloq_Margem','Disponivel_Secundario','FormaCondominio','FinanceiroFinal','Movimentacao_Solicitada','ResgTot','DataMov','DataCot','DataFin','StatusCRM']].drop_duplicates(subset=['NomeProduto','QuantidadeFinal','FinanceiroFinal','Movimentacao_Solicitada','StatusCRM']).reset_index(drop=True)
verificacao['Ação_PM'] = verificacao.apply(lambda row : acao_pm(row['NomeProduto'],row['FormaCondominio'],row['Disponivel_Secundario'],row['Movimentacao_Solicitada'],row['TipoProduto']),axis=1)
verificacao['Ação_PM'].loc[(verificacao['Classe']=='Crédito Alternativo')]='Verificar se podemos passar para PF'
verificacao['Ação_PM'].loc[(verificacao['NomeProduto']=='FIC FIM Crédito Privado - VIC CPHY (BRL)')]='Verificar se podemos passar para PF'
verificacao['Ação_PM'].loc[(verificacao['Disponivel_Secundario']!=np.nan)&(verificacao['ResgTot']==True)] = 'Ação Já Tomada'
verificacao['Ação_PM'].loc[(verificacao['Ação_PM']=='Disponibilizar no secundário')&(verificacao['TipoProduto']=='FIP')] = 'Verificar se podemos passar para PF'
verificacao['Ação_PM'].loc[(verificacao['Ação_PM']=='Disponibilizar no secundário')&(verificacao['TipoProduto']=='COTAS')] = 'Verificar se podemos passar para PF'
verificacao = verificacao.drop(columns=['ContaCRMGuid','AtivoGuid'])
    

In [169]:
#Clientes que precisam de conta no BTG

precisam_conta = verificacao[(verificacao['Ação_PM']=='Verificar se podemos passar para PF')].reset_index()
precisam_conta = precisam_conta[['NomeContaCRM','NomeProduto','TipoProduto','Conta_BTG','Officer','Controller','DPM']]
precisam_conta['Necessário_Abrir_BTG'] = np.nan
precisam_conta['Necessário_Abrir_BTG'].loc[(precisam_conta['TipoProduto']=='FIP')&(precisam_conta['Conta_BTG'].isnull())]='Abrir BTG'
precisam_conta=precisam_conta[precisam_conta['Necessário_Abrir_BTG']=='Abrir BTG'][['NomeContaCRM','Necessário_Abrir_BTG']].drop_duplicates()

precisam_conta_c = verificacao[verificacao['Possui Estrutura de Contas'].isnull()][['NomeContaCRM']].drop_duplicates()
precisam_conta_c['Necessário_Abrir_Conta'] = 'precisa da estrutura de contas'
precisam_conta = pd.concat([precisam_conta,precisam_conta_c])

In [170]:
#verificar fundos aptos a serem encerrados
prazos = ['D+0','D+1','C+1','D+2','C+0']
verificar_aptos_encerramento = verificacao[['NomeContaCRM','TipoProduto','NomeProduto','Classe','Conta_BTG','QuantidadeFinal','FinanceiroFinal','ResgCot','ResgTot','Ação_PM']].drop_duplicates()

#Retorna os produtos que são liquidos ou PA que estão ok para transferencia
produtosLiquidos_IliquidosOk = verificar_aptos_encerramento.loc[(verificar_aptos_encerramento['ResgCot'].isin(prazos))&(verificar_aptos_encerramento['Ação_PM']=='Resgatar / Vender')|(verificar_aptos_encerramento['Ação_PM']=='Ação Já Tomada')|((verificar_aptos_encerramento['TipoProduto']=='FIP')&(verificar_aptos_encerramento['Conta_BTG']=='Sm'))].dropna(subset=['NomeContaCRM'])
produtosLiquidos_IliquidosOk = produtosLiquidos_IliquidosOk.groupby(['NomeContaCRM']).count()[['NomeProduto']].reset_index().rename(columns={'NomeProduto':'Produtos_ok_seguir'})

#Total de produtos
produtos_total = verificar_aptos_encerramento.groupby(['NomeContaCRM']).count()[['NomeProduto']].reset_index().rename(columns={'NomeProduto':'Total_Produtos'})

verificar_aptos_encerramento  = pd.merge(left=produtos_total,right=produtosLiquidos_IliquidosOk,on='NomeContaCRM',how='left')
verificar_aptos_encerramento = verificar_aptos_encerramento[['NomeContaCRM','Total_Produtos','Produtos_ok_seguir']]
verificar_aptos_encerramento['Apto_Encerramento'] = verificar_aptos_encerramento['Total_Produtos']==verificar_aptos_encerramento['Produtos_ok_seguir']
verificar_aptos_encerramento['Apto_Encerramento'].loc[(verificar_aptos_encerramento['Apto_Encerramento']==True)]='Apto'
verificar_aptos_encerramento['Apto_Encerramento'].loc[(verificar_aptos_encerramento['Apto_Encerramento']==False)]= np.nan


In [13]:
#Separar os ativos de crédito
Ativos_credito = verificacao[['NomeContaCRM','NomeProduto','GuidProduto','TipoProduto','QuantidadeFinal','FinanceiroFinal','Ação_PM']].drop_duplicates()
produtos = crm.rating_produtos()
produtos['GuidProduto'] =produtos['GuidProduto'].str.lower() 
Ativos_credito = Ativos_credito.loc[~verificacao['TipoProduto'].isin(ativos_simples)]
Ativos_credito = Ativos_credito[Ativos_credito['Ação_PM']=='Disponibilizar no secundário']
Ativos_credito = pd.merge(left=Ativos_credito,right=produtos,on='GuidProduto',how='left' ).drop_duplicates()
Ativos_credito = Ativos_credito.groupby(['GuidProduto','NomeProduto','TipoProduto','CETIP','ISIN','Rating','IdPAS']).sum().reset_index()[['NomeProduto','TipoProduto','QuantidadeFinal','FinanceiroFinal','CETIP','ISIN','Rating','IdPAS']].sort_values(by=['NomeProduto'])
Ativos_credito = Ativos_credito[(Ativos_credito['TipoProduto']!='COTAS')&(Ativos_credito['TipoProduto']!='FIP')]
Ativos_credito = Ativos_credito.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Encerramento de Fundos/Ativos_Crédito.xlsx')

In [14]:

#Gerando excel para e-mail:
with pd.ExcelWriter('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Encerramento de Fundos/fundos_encerrados_automatico.xlsx') as writer:  
    verificacao.to_excel(writer, sheet_name='carteiras')
    verificar_aptos_encerramento.to_excel(writer, sheet_name='Fundos prontos') 
    precisam_conta.to_excel(writer, sheet_name='Verificar Conta') 
    

In [15]:
#Envio do e-mail para PM
#Puxa os emails dos destinatários responsáveis pela carteira.

fundos_pm = verificacao[verificacao['DPM']=='NA']
fundos_pm = fundos_pm[['GuidContaCRM','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','CodigoCetip','FormaCondominio','Officer','Controller','Ação_PM','ContaPE']].rename(columns={'ContaPE':'Possui_Conta_BTG'}).reset_index(drop=True)
fundos_pm=fundos_pm[fundos_pm['Ação_PM']!='Ação Já Tomada'].drop_duplicates()
fundos_pm.to_excel(f'C:/Temp/Foundation/Encerramento de Fundos/Officer/PM.xlsx')
#emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]

txt_inicio = f'''Prezados,<br>   
Peço analisar as ações necessárias para que o encerramento do fundo ocorra.<br><br>'''
subject= f'Fundos em encerramento'
to=['tamara.alves@jbfo.com','portfolio@jbfo.com','andre.nemi@juliusbaer.com','luiza.gomes@juliusbaer.com','mariana.drumond@juliusbaer.com']
email = Email(to = to , subject = subject, text= txt_inicio,send = False,attachments = f'C:/Temp/Foundation/Encerramento de Fundos/Officer/PM.xlsx')        

KeyError: "['ContaPE'] not in index"

In [ ]:
#Envio do e-mail para o comercial
#Puxa os emails dos destinatários responsáveis pela carteira.

fundos_c_dpm = verificacao[verificacao['DPM']!='NA']
officers = list(fundos_c_dpm ['Officer'].drop_duplicates())
fundos_c_dpm = fundos_c_dpm[['NomeContaCRM','NomeProduto','CodigoCetip','QuantidadeFinal','Disponivel_Secundario','FinanceiroFinal','DPM','Officer','Administrador','Ação_PM']] 
fundos_c_dpm = pd.merge(left=fundos_c_dpm, right=carteiras,on='NomeContaCRM',how='left')


for officer in officers:
    df = fundos_c_dpm[fundos_c_dpm['Officer']==officer]
    emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
    emails=list(set(emails))
    emails = [x for x in emails if x is not None]
    df = df.drop(columns=['emails_aprovadores'])
    df = df.drop_duplicates()
    df.to_excel(f'C:/Temp/Foundation/Encerramento de Fundos/Officer/{officer}.xlsx')
    #emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
  
    
    txt_inicio = f'''Prezados,<br>   
    Peço analisar as ações necessárias para que o encerramento do fundo ocorra.<br><br>'''
    subject= f'Fundos em encerramento'
    to=['tamara.alves@jbfo.com','portfolio@jbfo.com']+emails
    email = Email(to = to , subject = subject, text= txt_inicio,send = False,attachments = f'C:/Temp/Foundation/Encerramento de Fundos/Officer/{officer}.xlsx')        

In [ ]:
#Verificar os termos necessários e solicitar ao comercial
solicitar_termos = po_posicao[['NomeContaCRM','Cotista','Perfil','Officer','Controller']].drop_duplicates().rename(columns={'Perfil':'Portfolio'})
officers = list(solicitar_termos['Officer'].drop_duplicates())
termos_necessarios = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Encerramento de Fundos/Termos por mandato.xlsx')
solicitar_termos = pd.merge(left=solicitar_termos, right=termos_necessarios, on='Portfolio',how='left')
solicitar_termos = solicitar_termos[solicitar_termos['Termos'].notnull()]
solicitar_termos = solicitar_termos.drop_duplicates()
solicitar_termos = pd.merge(left=solicitar_termos,right=carteiras,on='NomeContaCRM',how='left').drop(columns=['NomeContaCRM'])

#Envio do email do termo
for officer in officers:
    df = solicitar_termos[solicitar_termos['Officer']==officer]
    emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
    emails=list(set(emails))
    emails = [x for x in emails if x is not None]
    df = df.drop(columns=['emails_aprovadores'])
    df = df.drop_duplicates()
    df.to_excel(f'C:/Temp/Foundation/Encerramento de Fundos/Officer/termos_{officer}.xlsx')
    #emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
  
    
    txt_inicio = f'''Prezados,<br>   
    Pedimos que solicitem os termos para que os cotistas não fiquem sem alocações nas classes.<br><br>
    PM ficará responsável pelo .<br><br>
    '''
    subject= f'Fundos em encerramento'
    to=['tamara.alves@jbfo.com','portfolio@jbfo.com']+emails
    email = Email(to = to , subject = subject, text= txt_inicio,send = False,attachments = f'C:/Temp/Foundation/Encerramento de Fundos/Officer/termos_{officer}.xlsx')        